# Import the necessary libraries


In [304]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns


from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    train_test_split,
    GridSearchCV,
)
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    RocCurveDisplay,
)
import warnings

warnings.filterwarnings("ignore")

# Exploring the dataset


## 1. Freezer data


In [305]:
freezer_data_df = pd.read_csv("./dataset/freezer_data.csv")
freezer_data_df.head()

,Freezer Name,Model Number,Volume Capacity (Liters),Power and maitainance Cost (LKR) per 100 hours
0,ChillMaster,M001,55,2500
1,FreezeZone,M002,75,2800
2,CoolTech,M003,120,3200
3,mini CoolTech,M004,30,1800
4,IceBlast Pro,M005,150,3500


So there are 10 types of freezers. We need to cluster the shops and recommend the best freezer for each cluster.


In [306]:
freezer_data_df.shape

(10, 4)

In [307]:
freezer_data_df.head(10)

,Freezer Name,Model Number,Volume Capacity (Liters),Power and maitainance Cost (LKR) per 100 hours
0,ChillMaster,M001,55,2500
1,FreezeZone,M002,75,2800
2,CoolTech,M003,120,3200
3,mini CoolTech,M004,30,1800
4,IceBlast Pro,M005,150,3500
5,IceBlast,M006,65,2700
6,FreezeTech,M007,100,3000
7,FreezeTech 3.0,M008,25,1600
8,ChillTech,M009,80,2800
9,IceTech,M010,200,4500


In [308]:
freezer_data_df.isna().sum()

Freezer Name                                      0
Model Number                                      0
Volume Capacity (Liters)                          0
Power and maitainance Cost (LKR) per 100 hours    0
dtype: int64

## 2. Outlet Area data


In [309]:
outlets_area_df = pd.read_csv("./dataset/outlets_data.csv")
outlets_area_df.head()

,Outlet_ID,area(sqft)
0,ID7203,5660
1,ID1878,4138
2,ID1740,5546
3,ID4366,5134
4,ID8358,4100


In [310]:
outlets_area_df.shape

(988, 2)

In [311]:
outlets_area_df["Outlet_ID"].nunique()

951

In [312]:
# find the duplicated outlets id
# get the duplicated outlets id, and the duplicated rows
duplicated_ids = (
    outlets_area_df[outlets_area_df["Outlet_ID"].duplicated()]["Outlet_ID"]
    .unique()
    .tolist()
)

In [313]:
for dup_id in duplicated_ids:
    print(f"Outlet ID: {dup_id}")
    print(outlets_area_df[outlets_area_df["Outlet_ID"] == dup_id])
    print("-" * 20)

Outlet ID: ID8278
    Outlet_ID  area(sqft)
320    ID8278        3600
384    ID8278        4980
--------------------
Outlet ID: ID8631
    Outlet_ID  area(sqft)
25     ID8631        4024
406    ID8631        3196
--------------------
Outlet ID: ID4003
    Outlet_ID  area(sqft)
77     ID4003        4841
444    ID4003        2990
--------------------
Outlet ID: ID1365
    Outlet_ID  area(sqft)
98     ID1365        5823
494    ID1365        1291
--------------------
Outlet ID: ID2170
    Outlet_ID  area(sqft)
310    ID2170        8077
525    ID2170        6169
--------------------
Outlet ID: ID1485
    Outlet_ID  area(sqft)
292    ID1485        4535
537    ID1485        9483
--------------------
Outlet ID: ID9465
    Outlet_ID  area(sqft)
441    ID9465        1583
563    ID9465        6650
--------------------
Outlet ID: ID6214
    Outlet_ID  area(sqft)
232    ID6214        1608
564    ID6214        4052
--------------------
Outlet ID: ID3549
    Outlet_ID  area(sqft)
415    ID3549       

In [314]:
outlets_area_df.isna().sum()

Outlet_ID     0
area(sqft)    0
dtype: int64

## 3. Product data


In [315]:
product_data_df = pd.read_csv("./dataset/product_data.csv")
product_data_df.head()

,pid,volume,product_name,price
0,IP1,0.2,Vanilla Mini Cone,100
1,IP2,0.2,Chocolate Petite Bar,90
2,IP3,0.2,Strawberry Tiny Cup,110
3,IP4,0.2,Mint Bite-size Pop,100
4,IP5,0.2,Butter Pecan Small Stick,100


In [316]:
product_data_df.shape

(10, 4)

In [317]:
product_data_df.head(10)

,pid,volume,product_name,price
0,IP1,0.2,Vanilla Mini Cone,100
1,IP2,0.2,Chocolate Petite Bar,90
2,IP3,0.2,Strawberry Tiny Cup,110
3,IP4,0.2,Mint Bite-size Pop,100
4,IP5,0.2,Butter Pecan Small Stick,100
5,BP1,1.0,Chocolate Fudge Big Scoop,1200
6,BP2,1.0,Moose Tracks Jumbo Cup,1100
7,BP3,1.0,Mint Chocolate Chip Family Pack,1150
8,BP4,1.0,Rocky Road Mega Tub,1000
9,BP5,1.0,Cookies and Cream Mega Tub,1100


In [318]:
product_data_df.isna().sum()

pid             0
volume          0
product_name    0
price           0
dtype: int64

## 4. Sales data


In [319]:
sales_data_df = pd.read_csv("./dataset/sales_data.csv")
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units
0,ID7203,1,IP1,Vanilla Mini Cone,874.0
1,ID7203,1,IP2,Chocolate Petite Bar,105.0
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0
3,ID7203,1,IP4,Mint Bite-size Pop,502.0
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0


In [320]:
sales_data_df.shape

(81000, 5)

In [321]:
sales_data_df["Outlet_ID"].nunique()

951

In [322]:
sales_data_df["product_name"].nunique()

10

In [323]:
sales_data_df["week"].nunique()

12

In [324]:
sales_data_df.describe()

,week,no_units
count,81000.000000,81000.000000
mean,6.504432,373.143543
std,3.453426,370.546966
min,1.000000,20.000000
25%,4.000000,79.000000
50%,7.000000,210.000000
75%,10.000000,593.000000
max,12.000000,1499.000000


In [325]:
sales_data_df.isna().sum()

Outlet_ID       0
week            0
pid             0
product_name    0
no_units        0
dtype: int64

## 5. Week data


In [326]:
week_data_df = pd.read_csv("./dataset/week_data.csv")
week_data_df.head()

,Week,Start Date,End Date
0,Week 1,1/2/2023,1/8/2023
1,Week 2,1/9/2023,1/15/2023
2,Week 3,1/16/2023,1/22/2023
3,Week 4,1/23/2023,1/29/2023
4,Week 5,1/30/2023,2/5/2023


In [327]:
week_data_df.shape

(12, 3)

In [328]:
week_data_df.head(12)

,Week,Start Date,End Date
0,Week 1,1/2/2023,1/8/2023
1,Week 2,1/9/2023,1/15/2023
2,Week 3,1/16/2023,1/22/2023
3,Week 4,1/23/2023,1/29/2023
4,Week 5,1/30/2023,2/5/2023
5,Week 6,2/6/2023,2/12/2023
6,Week 7,2/13/2023,2/19/2023
7,Week 8,2/20/2023,2/26/2023
8,Week 9,2/27/2023,3/5/2023
9,Week 10,3/6/2023,3/12/2023


Converting to the number to join with the sales data


In [329]:
week_data_df["week"] = week_data_df["Week"].str.replace("Week ", "").astype(int)

In [330]:
week_data_df.drop("Week", axis=1, inplace=True)

### Note:


We do have the data for three months <br>
We have weekly sales details as well. <br>


Extract the day and the month from the date and add it to the sales data. <br>
Then we can merge it with our own dataframes. <br>


There are some special days in these three months. <br>

-   1st Jan 2023
-   15th Jan 2023
-   14th Feb 2023


### Extracting the day and the month from the date


In [331]:
week_data_df.dtypes

Start Date    object
End Date      object
week           int32
dtype: object

In [332]:
week_data_df["Start Date"] = pd.to_datetime(week_data_df["Start Date"])

In [333]:
# convert the 'End Date' column to a datetime format
week_data_df["End Date"] = pd.to_datetime(week_data_df["End Date"])

# extract the week of the transaction and store it in a new column
week_data_df["month_of_transaction"] = week_data_df["End Date"].dt.month

week_data_df.head()

,Start Date,End Date,week,month_of_transaction
0,2023-01-02,2023-01-08,1,1
1,2023-01-09,2023-01-15,2,1
2,2023-01-16,2023-01-22,3,1
3,2023-01-23,2023-01-29,4,1
4,2023-01-30,2023-02-05,5,2


# Compiling and extracting information from the datasets


## Dataframe with unique outlet id's


In [334]:
sales_data_df["Outlet_ID"].nunique()

951

In [335]:
# create a dataframe with one column containing all the unique outlet ids
outlet_info_df = pd.DataFrame(
    sales_data_df["Outlet_ID"].unique(), columns=["Outlet_ID"]
)
outlet_info_df.head()

,Outlet_ID
0,ID7203
1,ID1878
2,ID1740
3,ID4366
4,ID8358


In [336]:
outlet_info_df.shape

(951, 1)

## Extracting info form the other datasets


We have the following dataframes now:

-   freezer_data_df : Freezer data (10 unique freezers)
-   outlets_area_df : Area of each outlet (there are 37 duplicates!!) & contains -> [outlet_id, area]
-   product_data_df : Product data (10 unique products) & contains -> [pid, volume, product_name, price]
-   sales_data_df : Sales data (contains 3 months data) & contains -> ['Outlet_ID', 'week', 'pid', 'product_name', 'no_units']
-   week_data_df : Information on 12 weeks & contains -> ['Start Date', 'End Date', 'week', 'month_of_transaction']


### Merging the dataframes

In [337]:
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units
0,ID7203,1,IP1,Vanilla Mini Cone,874.0
1,ID7203,1,IP2,Chocolate Petite Bar,105.0
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0
3,ID7203,1,IP4,Mint Bite-size Pop,502.0
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0


In [338]:
sales_data_df[["pid", "product_name"]].nunique()

pid             10
product_name    10
dtype: int64

In [339]:
# join the product data price and volume colume to sales_data_df
sales_data_df = sales_data_df.merge(
    product_data_df[["product_name", "price", "volume"]], on="product_name", how="left"
)
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units,price,volume
0,ID7203,1,IP1,Vanilla Mini Cone,874.0,100,0.2
1,ID7203,1,IP2,Chocolate Petite Bar,105.0,90,0.2
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0,110,0.2
3,ID7203,1,IP4,Mint Bite-size Pop,502.0,100,0.2
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0,100,0.2


In [340]:
sales_data_df = sales_data_df.merge(
    week_data_df[["Start Date", "End Date", "week", "month_of_transaction"]], on="week", how="left"
)
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units,price,volume,Start Date,End Date,month_of_transaction
0,ID7203,1,IP1,Vanilla Mini Cone,874.0,100,0.2,2023-01-02,2023-01-08,1
1,ID7203,1,IP2,Chocolate Petite Bar,105.0,90,0.2,2023-01-02,2023-01-08,1
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0,110,0.2,2023-01-02,2023-01-08,1
3,ID7203,1,IP4,Mint Bite-size Pop,502.0,100,0.2,2023-01-02,2023-01-08,1
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0,100,0.2,2023-01-02,2023-01-08,1


### Note:

At this point we have the following dataframes:
- sales_data_df
- outlets_area_df
- freezer_data_df

outlets_area_df has 37 duplicates. We need to remove them. <br>
Since, we don't which one to remove, yet we will keep all of them. <br>
We can make the decision once we have the final dataframe. <br>
<br>
For the Freezer_data_df it contains the freezers and its information. So we have to assign a freezer to a shop at last. Clusster the shops into 10 categories and assign the freezer to each cluster. <br>

## Creating more features in the sales_data_df

### Get total earnings per row

In [341]:
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units,price,volume,Start Date,End Date,month_of_transaction
0,ID7203,1,IP1,Vanilla Mini Cone,874.0,100,0.2,2023-01-02,2023-01-08,1
1,ID7203,1,IP2,Chocolate Petite Bar,105.0,90,0.2,2023-01-02,2023-01-08,1
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0,110,0.2,2023-01-02,2023-01-08,1
3,ID7203,1,IP4,Mint Bite-size Pop,502.0,100,0.2,2023-01-02,2023-01-08,1
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0,100,0.2,2023-01-02,2023-01-08,1


In [342]:
sales_data_df['total_earning'] = sales_data_df['price'] * sales_data_df['no_units']
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units,price,volume,Start Date,End Date,month_of_transaction,total_earning
0,ID7203,1,IP1,Vanilla Mini Cone,874.0,100,0.2,2023-01-02,2023-01-08,1,87400.0
1,ID7203,1,IP2,Chocolate Petite Bar,105.0,90,0.2,2023-01-02,2023-01-08,1,9450.0
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0,110,0.2,2023-01-02,2023-01-08,1,131780.0
3,ID7203,1,IP4,Mint Bite-size Pop,502.0,100,0.2,2023-01-02,2023-01-08,1,50200.0
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0,100,0.2,2023-01-02,2023-01-08,1,71000.0


### Get total volume sold per row

In [343]:
sales_data_df['total_volume_sold'] = sales_data_df['volume'] * sales_data_df['no_units']
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units,price,volume,Start Date,End Date,month_of_transaction,total_earning,total_volume_sold
0,ID7203,1,IP1,Vanilla Mini Cone,874.0,100,0.2,2023-01-02,2023-01-08,1,87400.0,174.8
1,ID7203,1,IP2,Chocolate Petite Bar,105.0,90,0.2,2023-01-02,2023-01-08,1,9450.0,21.0
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0,110,0.2,2023-01-02,2023-01-08,1,131780.0,239.6
3,ID7203,1,IP4,Mint Bite-size Pop,502.0,100,0.2,2023-01-02,2023-01-08,1,50200.0,100.4
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0,100,0.2,2023-01-02,2023-01-08,1,71000.0,142.0


## Adding new features to the final dataframe

In [344]:
outlet_info_df.head()

,Outlet_ID
0,ID7203
1,ID1878
2,ID1740
3,ID4366
4,ID8358


In [345]:
sales_data_df.head()

,Outlet_ID,week,pid,product_name,no_units,price,volume,Start Date,End Date,month_of_transaction,total_earning,total_volume_sold
0,ID7203,1,IP1,Vanilla Mini Cone,874.0,100,0.2,2023-01-02,2023-01-08,1,87400.0,174.8
1,ID7203,1,IP2,Chocolate Petite Bar,105.0,90,0.2,2023-01-02,2023-01-08,1,9450.0,21.0
2,ID7203,1,IP3,Strawberry Tiny Cup,1198.0,110,0.2,2023-01-02,2023-01-08,1,131780.0,239.6
3,ID7203,1,IP4,Mint Bite-size Pop,502.0,100,0.2,2023-01-02,2023-01-08,1,50200.0,100.4
4,ID7203,1,IP5,Butter Pecan Small Stick,710.0,100,0.2,2023-01-02,2023-01-08,1,71000.0,142.0


### 1. Units sold per week

In [346]:
# group by Outlet_ID and week, and get the sum of no_units sold per week
weekly_sales_df = sales_data_df.groupby(["Outlet_ID", "week"])['no_units'].sum().reset_index()
weekly_sales_df.head()

,Outlet_ID,week,no_units
0,ID1008,1,3990.0
1,ID1008,2,3293.0
2,ID1008,3,3590.0
3,ID1008,4,2691.0
4,ID1008,5,1522.0


In [347]:
# for each shop plot the no_units sold per week
# for outlet_id in weekly_sales_df["Outlet_ID"].unique():
#     outlet_weekly_sales_df = weekly_sales_df[weekly_sales_df["Outlet_ID"] == outlet_id]
#     plt.figure(figsize=(15, 5))
#     plt.plot(outlet_weekly_sales_df["week"], outlet_weekly_sales_df["no_units"])
#     plt.title(f"Outlet ID: {outlet_id}")
#     plt.xlabel("Week")
#     plt.ylabel("No. of units sold")
#     plt.show()

In [348]:
for week in range(1, 13):
    week_sales_df = weekly_sales_df[weekly_sales_df["week"] == week].reset_index(drop=True)
    week_sales_df.drop("week", axis=1, inplace=True)

    week_sales_df.rename(columns={"no_units": f"weekly_sale_for_{week}"}, inplace=True)
    # merge the week_sales_df with outlet_info_df
    outlet_info_df = outlet_info_df.merge(week_sales_df, on="Outlet_ID", how="left")

In [349]:
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,weekly_sale_for_10,weekly_sale_for_11,weekly_sale_for_12
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,2484.0,4382.0,2233.0
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,3961.0,2210.0,2469.0
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,3081.0,3693.0,3067.0
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,4975.0,2426.0,3579.0
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,2831.0,2338.0,4071.0


### 2. Total volume sold per week

In [350]:
weekly_volume_df = sales_data_df.groupby(["Outlet_ID", "week"])['total_volume_sold'].sum().reset_index()
weekly_volume_df.head()

,Outlet_ID,week,total_volume_sold
0,ID1008,1,798.0
1,ID1008,2,658.6
2,ID1008,3,718.0
3,ID1008,4,538.2
4,ID1008,5,304.4


In [352]:
for week in range(1, 13):
    week_vol_df = weekly_volume_df[weekly_volume_df["week"] == week].reset_index(drop=True)
    week_vol_df.drop("week", axis=1, inplace=True)

    week_vol_df.rename(columns={"total_volume_sold": f"weekly_sold_volume_for_{week}"}, inplace=True)
    # merge the week_vol_df with outlet_info_df
    outlet_info_df = outlet_info_df.merge(week_vol_df, on="Outlet_ID", how="left")

In [353]:
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,weekly_sold_volume_for_3,weekly_sold_volume_for_4,weekly_sold_volume_for_5,weekly_sold_volume_for_6,weekly_sold_volume_for_7,weekly_sold_volume_for_8,weekly_sold_volume_for_9,weekly_sold_volume_for_10,weekly_sold_volume_for_11,weekly_sold_volume_for_12
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,561.0,516.6,560.4,645.4,598.4,627.6,461.8,496.8,876.4,446.6
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,561.8,744.8,642.6,662.6,576.6,700.2,662.4,792.2,442.0,493.8
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,740.4,612.0,764.4,638.0,427.6,562.4,414.4,616.2,738.6,613.4
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,768.6,807.2,465.6,490.8,636.2,605.0,795.0,995.0,485.2,715.8
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,686.6,910.4,216.0,719.6,632.8,699.4,747.6,566.2,467.6,814.2


### 3. Total earnings per week

In [354]:
weekly_earning_df = sales_data_df.groupby(["Outlet_ID", "week"])['total_earning'].sum().reset_index()
weekly_earning_df.head()

,Outlet_ID,week,total_earning
0,ID1008,1,397290.0
1,ID1008,2,332330.0
2,ID1008,3,362450.0
3,ID1008,4,273290.0
4,ID1008,5,155790.0


In [355]:
for week in range(1, 13):
    week_earning_df = weekly_earning_df[weekly_earning_df["week"] == week].reset_index(drop=True)
    week_earning_df.drop("week", axis=1, inplace=True)

    week_earning_df.rename(columns={"total_earning": f"weekly_earning_for_{week}"}, inplace=True)
    # merge the week_earning_df with outlet_info_df
    outlet_info_df = outlet_info_df.merge(week_earning_df, on="Outlet_ID", how="left")

In [356]:
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,weekly_earning_for_3,weekly_earning_for_4,weekly_earning_for_5,weekly_earning_for_6,weekly_earning_for_7,weekly_earning_for_8,weekly_earning_for_9,weekly_earning_for_10,weekly_earning_for_11,weekly_earning_for_12
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,275200.0,259650.0,271830.0,327390.0,301010.0,320700.0,227830.0,241650.0,437660.0,231560.0
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,282580.0,379070.0,322280.0,330430.0,277250.0,350880.0,330170.0,401670.0,225080.0,237880.0
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,369650.0,300560.0,385470.0,316730.0,215310.0,277610.0,211340.0,312190.0,370450.0,303370.0
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,383500.0,404840.0,224220.0,254280.0,321540.0,300490.0,408690.0,497280.0,243100.0,362090.0
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,340710.0,454070.0,101010.0,354670.0,309570.0,347220.0,383510.0,282780.0,233040.0,403370.0


### 4. No of units sold for each product

In [358]:
feature_cnt_df = (
    sales_data_df.groupby(["Outlet_ID", "product_name"])["no_units"]
    .sum()
    .reset_index()
)
feature_cnt_df.head()

,Outlet_ID,product_name,no_units
0,ID1008,Butter Pecan Small Stick,6055.0
1,ID1008,Chocolate Petite Bar,7206.0
2,ID1008,Mint Bite-size Pop,6486.0
3,ID1008,Strawberry Tiny Cup,6285.0
4,ID1008,Vanilla Mini Cone,6970.0


In [359]:
for product in feature_cnt_df['product_name'].unique():
    product_df = feature_cnt_df[feature_cnt_df['product_name'] == product].reset_index(drop=True)
    product_df.drop('product_name', axis=1, inplace=True)
    product_df.rename(columns={'no_units': f'{product}_units_sold'}, inplace=True)
    outlet_info_df = outlet_info_df.merge(product_df, on='Outlet_ID', how='left')

In [360]:
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Butter Pecan Small Stick_units_sold,Chocolate Petite Bar_units_sold,Mint Bite-size Pop_units_sold,Strawberry Tiny Cup_units_sold,Vanilla Mini Cone_units_sold,Chocolate Fudge Big Scoop_units_sold,Cookies and Cream Mega Tub_units_sold,Mint Chocolate Chip Family Pack_units_sold,Moose Tracks Jumbo Cup_units_sold,Rocky Road Mega Tub_units_sold
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,6443.0,7295.0,5520.0,9127.0,6361.0,NaN,NaN,NaN,NaN,NaN
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,6748.0,6622.0,7954.0,6856.0,7860.0,NaN,NaN,NaN,NaN,NaN
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,6817.0,8116.0,7192.0,7837.0,7625.0,NaN,NaN,NaN,NaN,NaN
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,9524.0,6259.0,8759.0,9232.0,6530.0,NaN,NaN,NaN,NaN,NaN
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,8866.0,9562.0,6241.0,7652.0,7057.0,NaN,NaN,NaN,NaN,NaN


In [361]:
# fill the missing values with 0
outlet_info_df.fillna(0, inplace=True)
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Butter Pecan Small Stick_units_sold,Chocolate Petite Bar_units_sold,Mint Bite-size Pop_units_sold,Strawberry Tiny Cup_units_sold,Vanilla Mini Cone_units_sold,Chocolate Fudge Big Scoop_units_sold,Cookies and Cream Mega Tub_units_sold,Mint Chocolate Chip Family Pack_units_sold,Moose Tracks Jumbo Cup_units_sold,Rocky Road Mega Tub_units_sold
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,6443.0,7295.0,5520.0,9127.0,6361.0,0.0,0.0,0.0,0.0,0.0
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,6748.0,6622.0,7954.0,6856.0,7860.0,0.0,0.0,0.0,0.0,0.0
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,6817.0,8116.0,7192.0,7837.0,7625.0,0.0,0.0,0.0,0.0,0.0
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,9524.0,6259.0,8759.0,9232.0,6530.0,0.0,0.0,0.0,0.0,0.0
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,8866.0,9562.0,6241.0,7652.0,7057.0,0.0,0.0,0.0,0.0,0.0


### 5. Total volume sold for each product

In [364]:
feature_vol_df = (
    sales_data_df.groupby(["Outlet_ID", "product_name"])["total_volume_sold"]
    .sum()
    .reset_index()
)
feature_vol_df.head()

,Outlet_ID,product_name,total_volume_sold
0,ID1008,Butter Pecan Small Stick,1211.0
1,ID1008,Chocolate Petite Bar,1441.2
2,ID1008,Mint Bite-size Pop,1297.2
3,ID1008,Strawberry Tiny Cup,1257.0
4,ID1008,Vanilla Mini Cone,1394.0


In [365]:
for product in feature_vol_df['product_name'].unique():
    product_df = feature_vol_df[feature_vol_df['product_name'] == product].reset_index(drop=True)
    product_df.drop('product_name', axis=1, inplace=True)
    product_df.rename(columns={'total_volume_sold': f'{product}_volume_sold'}, inplace=True)
    outlet_info_df = outlet_info_df.merge(product_df, on='Outlet_ID', how='left')

In [366]:
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Butter Pecan Small Stick_volume_sold,Chocolate Petite Bar_volume_sold,Mint Bite-size Pop_volume_sold,Strawberry Tiny Cup_volume_sold,Vanilla Mini Cone_volume_sold,Chocolate Fudge Big Scoop_volume_sold,Cookies and Cream Mega Tub_volume_sold,Mint Chocolate Chip Family Pack_volume_sold,Moose Tracks Jumbo Cup_volume_sold,Rocky Road Mega Tub_volume_sold
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,1288.6,1459.0,1104.0,1825.4,1272.2,NaN,NaN,NaN,NaN,NaN
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,1349.6,1324.4,1590.8,1371.2,1572.0,NaN,NaN,NaN,NaN,NaN
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,1363.4,1623.2,1438.4,1567.4,1525.0,NaN,NaN,NaN,NaN,NaN
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,1904.8,1251.8,1751.8,1846.4,1306.0,NaN,NaN,NaN,NaN,NaN
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,1773.2,1912.4,1248.2,1530.4,1411.4,NaN,NaN,NaN,NaN,NaN


In [367]:
outlet_info_df.fillna(0, inplace=True)
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Butter Pecan Small Stick_volume_sold,Chocolate Petite Bar_volume_sold,Mint Bite-size Pop_volume_sold,Strawberry Tiny Cup_volume_sold,Vanilla Mini Cone_volume_sold,Chocolate Fudge Big Scoop_volume_sold,Cookies and Cream Mega Tub_volume_sold,Mint Chocolate Chip Family Pack_volume_sold,Moose Tracks Jumbo Cup_volume_sold,Rocky Road Mega Tub_volume_sold
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,1288.6,1459.0,1104.0,1825.4,1272.2,0.0,0.0,0.0,0.0,0.0
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,1349.6,1324.4,1590.8,1371.2,1572.0,0.0,0.0,0.0,0.0,0.0
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,1363.4,1623.2,1438.4,1567.4,1525.0,0.0,0.0,0.0,0.0,0.0
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,1904.8,1251.8,1751.8,1846.4,1306.0,0.0,0.0,0.0,0.0,0.0
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,1773.2,1912.4,1248.2,1530.4,1411.4,0.0,0.0,0.0,0.0,0.0


### 6. Total earnings for each product

In [369]:
feature_earn_df = (
    sales_data_df.groupby(["Outlet_ID", "product_name"])["total_earning"]
    .sum()
    .reset_index()
)
feature_earn_df.head()

,Outlet_ID,product_name,total_earning
0,ID1008,Butter Pecan Small Stick,605500.0
1,ID1008,Chocolate Petite Bar,648540.0
2,ID1008,Mint Bite-size Pop,648600.0
3,ID1008,Strawberry Tiny Cup,691350.0
4,ID1008,Vanilla Mini Cone,697000.0


In [370]:
for product in feature_earn_df['product_name'].unique():
    product_df = feature_earn_df[feature_earn_df['product_name'] == product].reset_index(drop=True)
    product_df.drop('product_name', axis=1, inplace=True)
    product_df.rename(columns={'total_earning': f'{product}_earning'}, inplace=True)
    outlet_info_df = outlet_info_df.merge(product_df, on='Outlet_ID', how='left')

In [372]:
outlet_info_df.fillna(0, inplace=True)
outlet_info_df.head()

,Outlet_ID,weekly_sale_for_1,weekly_sale_for_2,weekly_sale_for_3,weekly_sale_for_4,weekly_sale_for_5,weekly_sale_for_6,weekly_sale_for_7,weekly_sale_for_8,weekly_sale_for_9,...,Butter Pecan Small Stick_earning,Chocolate Petite Bar_earning,Mint Bite-size Pop_earning,Strawberry Tiny Cup_earning,Vanilla Mini Cone_earning,Chocolate Fudge Big Scoop_earning,Cookies and Cream Mega Tub_earning,Mint Chocolate Chip Family Pack_earning,Moose Tracks Jumbo Cup_earning,Rocky Road Mega Tub_earning
0,ID7203,3389.0,2402.0,2805.0,2583.0,2802.0,3227.0,2992.0,3138.0,2309.0,...,644300.0,656550.0,552000.0,1003970.0,636100.0,0.0,0.0,0.0,0.0,0.0
1,ID1878,2597.0,2048.0,2809.0,3724.0,3213.0,3313.0,2883.0,3501.0,3312.0,...,674800.0,595980.0,795400.0,754160.0,786000.0,0.0,0.0,0.0,0.0,0.0
2,ID1740,3742.0,3208.0,3702.0,3060.0,3822.0,3190.0,2138.0,2812.0,2072.0,...,681700.0,730440.0,719200.0,862070.0,762500.0,0.0,0.0,0.0,0.0,0.0
3,ID4366,3410.0,3072.0,3843.0,4036.0,2328.0,2454.0,3181.0,3025.0,3975.0,...,952400.0,563310.0,875900.0,1015520.0,653000.0,0.0,0.0,0.0,0.0,0.0
4,ID8358,2891.0,4185.0,3433.0,4552.0,1080.0,3598.0,3164.0,3497.0,3738.0,...,886600.0,860580.0,624100.0,841720.0,705700.0,0.0,0.0,0.0,0.0,0.0
